## Preparación de los datos
Este notebook limpiara, y creara features iniciales

In [96]:
# settings
import pandas as pd
from   itertools import chain

In [7]:
# more settings
# import janytor

ModuleNotFoundError: No module named 'janytor'

In [2]:
# data
path = "work/data/celulares.csv"

In [3]:
import janitor

In [8]:
# functions
# corrigé un error en el formato de los valores de cada instancia
def df_csv(dataframe, string = ','):
  # elimina el caracter molesto
  df = dataframe.copy()
  # column by column
  for columna in df.columns.values:
    df[columna] = df[columna].str.replace(string, '')
  return df 
# lowercase all dataframe
def df_lowercase(dataframe):
  # lowercase all columns
  df = dataframe.copy()
  for columna in df.columns.values:
    df[columna] = df[columna].str.lower()
  return df
# coerse columns
def df_numeric(dataframe, columns):
  df = dataframe.copy()
  df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')
  return df
# agrupo las funciones anteriores
def df_clean(dataframe, string, columns_to_numeric):
  df = dataframe.copy()
  # 
  df_2 = df_csv(dataframe, string)
  df_3 = df_lowercase(df_2)
  df_4 = df_numeric(df_3, columns = columns_to_numeric)
  return df_4

In [4]:
# data loading
df_raw = pd.read_csv(path, dtype = 'str').clean_names()

In [6]:
# renombro columnas
nombres = {"nombre_del_producto": 'producto_nombre', 'memoria_interna': 'memoria', 'sistema_operativo': 'so'}
#
df_inicio = df_raw.rename(columns = nombres)

In [9]:
# limpieza
columns_numeric = ['peso', 'camara_trasera', 'camara_frontal', 'ram', 'memoria', 'precio']
# 
df_limpio = df_clean(df_inicio, ',', columns_numeric)

## Preparación de las caracteristicas

e.g so = {'android', 'ios'}

In [36]:
# elimino columnas
columnas_ignorar = {'producto_nombre', 'color', 'pantalla'}
df_seleccion = df_limpio.drop(columns = columnas_ignorar)

In [108]:
# dejaré solo la tecnología más reciente
# orden creciente 2g == gsm, 5g < lte ==4glte
pd.DataFrame({'tecnologia' : ['2g/3g/4g/4glte/5g', '4glte',  '4g/gsm', '2g/3g/4g/4glte/gsm', '4g', '5g', '3g/4g/gsm', '4g/4glte/gsm/lte', '2g/3g/lte', '3g/lte'],
             'tecnologia_mejor' : ['5g',            '4glte', '4g',     '4glte',              '4g', '5g', '4g',       '4glte',             '4glte',      '4glte']})

,tecnologia,tecnologia_mejor
0,2g/3g/4g/4glte/5g,5g
1,4glte,4glte
2,4g/gsm,4g
3,2g/3g/4g/4glte/gsm,4glte
4,4g,4g
5,5g,5g
6,3g/4g/gsm,4g
7,4g/4glte/gsm/lte,4glte
8,2g/3g/lte,lte
9,3g/lte,lte


In [71]:
# procesador
# df_seleccion.procesador.str.split().str.get(0).str.replace('\d+', '').unique()
# df_seleccion.so.str.extract(r'(android|ios)', expand = False)

0      android
1      android
2          ios
3          ios
4          ios
        ...   
130    android
131    android
132    android
133    android
134    android
Name: so, Length: 135, dtype: object

In [49]:
df_seleccion

,marca,peso,camara_trasera,camara_frontal,procesador,ram,memoria,so,tecnologia,precio
0,samsung,0.282,12,10,qualcomm1,12,256,android 10,2g/3g/4g/4glte/5g,46799
1,samsung,0.282,12,10,qualcomm1,12,256,android 10,2g/3g/4g/4glte/5g,46799
2,apple,0.272,12,12,apple1,4,512,apple ios 13,4glte,33999
3,apple,0.272,12,12,apple1,4,512,apple ios 13,4glte,33999
4,apple,0.272,12,12,apple1,4,512,apple ios 13,4glte,33999
...,...,...,...,...,...,...,...,...,...,...
130,motorola,0.188,8,13,mediatek2,2,32,android 9,4glte,3490
131,motorola,0.149,13,8,mediatek2,2,32,android 8,4glte,2990
132,huawei,0.176,13,8,mediatek2,2,32,android 9,4glte,3299
133,motorola,0.190,13,5,qualcomm3,1,16,android 8,4glte,2499
